In [1]:
#!pip install IProgress
import argparse

import torch

from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import pandas as pd
import scipy.sparse as sp
from utility.preprocessing import sparse_to_tensor, to_dense
from utility.rank_metrics import *
from utility.iofile import *
from sklearn.metrics import label_ranking_average_precision_score, mean_squared_error
import sys

In [2]:
import numpy as np
import scipy.sparse as sp
import pandas as pd
import torch
from sklearn.model_selection import ShuffleSplit
from utility.preprocessing import *
import time


In [4]:
#del sys.modules["model"]
from model import MedGCN, MultiMatLoss 

In [5]:
#del sys.modules["preparedata"]
from preparedata import * 

Preprocessing ...
reading adjacency matrix csv files ...
Making row orders consistent
constructing tensors of adjacency matrix ...
processing node features ...
number of encounters:  136647
number of providers:  5787
number of icd:  1676
number of icu:  182
number of encounter features:  136647
number of provider features:  5787
number of icd features:  1676
number of icu features:  182
encounter feature matrix shape: torch.Size([136647, 136647])
provider feature matrix shape: torch.Size([5787, 5787])
icd feature matrix shape: torch.Size([1676, 1676])
icu feature matrix shape: torch.Size([182, 182])
splitting train validation test sets ...


RuntimeError: CUDA out of memory. Tried to allocate 69.56 GiB (GPU 0; 14.76 GiB total capacity; 4.30 GiB already allocated; 9.98 GiB free; 4.31 GiB reserved in total by PyTorch)

In [88]:
#regularization parameter for lab error
args_lamb = 1
#GCN layers
args_layers = 1

In [89]:
print(fea_num_identity)
print(fea_num_node)

In [90]:
torch.manual_seed(666)

epochs=3
# n_iter_no_change=50

adj_losstype = { (0, 1): [('BCE',0)],   (0, 2): [('MSE',args_lamb)],   (0, 3): [('BCE',1)] }
print("tasks:",tasks)

medgcn =  MedGCN(fea_num_node, ( 300,  )*args_layers, 1, tasks, dropout=0.0).cuda()
optimizer = optim.Adam(medgcn.parameters(), lr=1e-3)

# change loss function to cross entropy
lossfun = nn.BCELoss()

# lossfun = MultiMatLoss(pos_weight=None).cuda()
# scheduler = ReduceLROnPlateau(optimizer, 'min')

tasks: ((0, 2),)


In [91]:
# training-validation-test
tr_loss, v_loss = [],[]
v_mapk, te_mapk = [],[]
v_lrap, te_lrap = [],[]
v_mse, te_mse = [],[]
best_val_loss = np.inf
best_val_lrap, best_val_mse = 0, np.inf
best_epoch = 0
for epoch in range(epochs):
    #training
    print("training............")
    medgcn.train()
    optimizer.zero_grad()
    y_train_pred, z = medgcn(train_fea_mats, train_adj_mats, train_adj_masks )
    #train_loss=lossfun(adj_recon, adj_mats, train_adj_masks, adj_losstype)

    train_loss = lossfun(y_train_pred[(0,2)][0], y_train.unsqueeze(1))
    train_loss.backward()
    optimizer.step()
    
    y_train_acc = (y_train_pred[(0,2)][0].reshape(-1).detach().cpu().numpy().round() == y_train.cpu().numpy()).sum() / y_train.cpu().numpy().shape[0]

    # validation-test
    print("validation............")
    medgcn.eval()
    with torch.no_grad():
        y_val_pred, z = medgcn(val_fea_mats, val_adj_mats, val_adj_masks )
        y_test_pred, z = medgcn(test_fea_mats, test_adj_mats, test_adj_masks )
                
        y_val_acc = (y_val_pred[(0,2)][0].reshape(-1).detach().cpu().numpy().round() == y_val.cpu().numpy()).sum() / y_val.cpu().numpy().shape[0]
        
        y_test_acc = (y_test_pred[(0,2)][0].reshape(-1).detach().cpu().numpy().round() == y_test.cpu().numpy()).sum() / y_test.cpu().numpy().shape[0]
        
        val_loss = lossfun(y_val_pred[(0,2)][0], y_val.unsqueeze(1))
        test_loss = lossfun(y_test_pred[(0,2)][0], y_test.unsqueeze(1))
    
    print('====> Epoch: {} train loss: {:.4f}, validation loss: {:.4f}, test loss: {:.4f}, training accuracy: {:.4f}, validation accuracy: {:.4f}, test accuracy: {:.4f}'  \
          .format(epoch, train_loss.item(), val_loss.item(), test_loss.item(),y_train_acc.item(), y_val_acc.item(), y_test_acc.item()))
    
    tr_loss.append(train_loss.item())
    v_loss.append(val_loss.item())
    
     
###### early stopping
#     if val_loss<=best_val_loss:
#         best_val_loss=val_loss
#         best_epoch=epoch
        
#     if test_mse<=best_val_mse:
#         best_val_mse=test_mse
#         save_obj(adj_recon[(0,2)][0].cpu().numpy(), 'est_lab2.pkl')
#         best_epoch=epoch
        
#     if test_lrap>=best_val_lrap:
#         best_val_lrap=test_lrap
#         save_obj(adj_recon[(0,3)][0].cpu().numpy(), 'rec_med2.pkl')
#         best_epoch=epoch

#     if epoch-best_epoch>n_iter_no_change:
#         break

training............
validation............
====> Epoch: 0 train loss: 2.8537, validation loss: 11.0532, test loss: 10.3549, training accuracy: 0.7500, validation accuracy: 0.6000, test accuracy: 0.5000
training............
validation............
====> Epoch: 1 train loss: 2.3038, validation loss: 11.0811, test loss: 13.8155, training accuracy: 0.9167, validation accuracy: 0.6000, test accuracy: 0.5000
training............
validation............
====> Epoch: 2 train loss: 2.3026, validation loss: 11.6180, test loss: 13.8155, training accuracy: 0.9167, validation accuracy: 0.4000, test accuracy: 0.5000
training............
validation............
====> Epoch: 3 train loss: 2.3026, validation loss: 15.5614, test loss: 13.8155, training accuracy: 0.9167, validation accuracy: 0.2000, test accuracy: 0.5000
training............
validation............
====> Epoch: 4 train loss: 2.3026, validation loss: 18.5493, test loss: 13.8155, training accuracy: 0.9167, validation accuracy: 0.2000, test ac

In [ ]:
res=pd.DataFrame({   'tr_loss': tr_loss, 'v_loss': v_loss, 
                  'v_mapk': v_mapk,'te_mapk': te_mapk, 'v_lrap': v_lrap, 'te_lrap': te_lrap,
                  'v_mse': v_mse, 'te_mse': te_mse
                 })
# res.to_csv('res_Med_lab%s.csv'%(args.lamb))

In [ ]:
res

In [109]:
y_test_pred, z = medgcn(test_fea_mats, test_adj_mats, test_adj_masks )

In [110]:
y_test_pred

{(0,
  2): [tensor([[0.9823],
          [0.8957],
          [0.9996],
          [0.9512],
          [0.9884]], device='cuda:0', grad_fn=<SigmoidBackward>)]}